## Workflow for Processing MCAS

In [1]:
import pandas as pd
from pathlib import Path
from mcasprocessor import MCASProcessor

pd.set_option('display.max_columns', 25)

In [2]:
filename = Path('sample-mcas.csv')

columns_of_interest = ['sasid', 'stugrade', 'eperf2', 'mperf2', 'sperf2', 'escaleds', 
                        'mscaleds', 'sscaleds', 'ecpi','mcpi', 'scpi']

subjects = ['ela', 'math', 'science']

output_column_format = ['NCESID', 'StudentTestID', 'StudentLocalID', 'StudentGradeLevel',
                        'TestDate', 'TestName', 'TestTypeName', 'TestSubjectName', 'TestGradeLevel',
                        'Score1Label', 'Score1Type', 'Score1Value',
                        'Score2Label', 'Score2Type', 'Score2Value',
                        'Score3Label', 'Score3Type', 'Score3Value',
                        'Score4Label', 'Score4Type', 'Score4Value'
                        ]

In [3]:
%time

time_full_df = pd.read_csv(filename)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 6.2 µs


In [4]:
%time

time_partial_df = pd.read_csv(filename, usecols=columns_of_interest)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.05 µs


In [5]:
# Keep df_original to compare with transformations
df = pd.read_csv(filename, usecols=columns_of_interest)
df_original = df.copy()
df

,sasid,stugrade,escaleds,eperf2,ecpi,mscaleds,mperf2,mcpi,sscaleds,sperf2,scpi
0,1408905,3,216,W,25,210,W,25,,,
1,1020285,3,238,NI,75,244,P,100,,,
2,1681155,3,220,NI,50,236,NI,75,,,
3,1987801,3,252,P,100,246,P,100,,,
4,1351732,3,246,P,100,228,NI,50,,,
...,...,...,...,...,...,...,...,...,...,...,...
2666,1769669,10,,F,100,,F,100,,F,75
2667,1757999,8,244,P,100,218,W,25,216,W,25
2668,1633264,10,236,NI,75,230,NI,75,222,NI,50
2669,1760810,7,,W,100,,W,100,,,


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2671 entries, 0 to 2670
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sasid     2671 non-null   int64 
 1   stugrade  2671 non-null   object
 2   escaleds  2671 non-null   object
 3   eperf2    2671 non-null   object
 4   ecpi      2671 non-null   object
 5   mscaleds  2671 non-null   object
 6   mperf2    2671 non-null   object
 7   mcpi      2671 non-null   object
 8   sscaleds  2671 non-null   object
 9   sperf2    2671 non-null   object
 10  scpi      2671 non-null   object
dtypes: int64(1), object(10)
memory usage: 229.7+ KB


In [7]:
for col in df.columns:
    print('COLUMN: ', col)
    print(df[col].value_counts())
    print()

COLUMN:  sasid
1840155    2
1603583    1
1619332    1
1140098    1
1555839    1
          ..
1659356    1
1129142    1
1043124    1
1187782    1
1445888    1
Name: sasid, Length: 2670, dtype: int64

COLUMN:  stugrade
3     389
4     380
5     378
8     377
6     358
7     357
10    339
9      59
11     25
12      8
SP      1
Name: stugrade, dtype: int64

COLUMN:  escaleds
       210
252    157
244    136
250    134
246    132
242    124
258    120
262    116
248    108
254     98
240     95
256     93
238     92
220     80
234     70
268     69
218     68
232     68
236     60
226     58
260     54
230     53
264     51
266     50
272     48
228     45
216     43
276     35
224     34
222     33
214     28
280     24
270     23
212     22
210     19
208      9
278      8
274      4
Name: escaleds, dtype: int64

COLUMN:  eperf2
P     1189
NI     589
A      481
W      232
       164
F       16
Name: eperf2, dtype: int64

COLUMN:  ecpi
100    1721
75      348
50      249
25      180
     

## Record with Duplicate `sasid`

Keeping both, why is this the only duplicate?

In [8]:
len(df.sasid.unique())

2670

In [9]:
df[df.sasid.duplicated()]

,sasid,stugrade,escaleds,eperf2,ecpi,mscaleds,mperf2,mcpi,sscaleds,sperf2,scpi
2450,1840155,10,268,A,100,264,A,100,262,A,100


In [10]:
df[df['sasid'] == 1840155]

,sasid,stugrade,escaleds,eperf2,ecpi,mscaleds,mperf2,mcpi,sscaleds,sperf2,scpi
1363,1840155,7,254,P,100,258,P,100,,,
2450,1840155,10,268,A,100,264,A,100,262,A,100


# Missing Test Scores

## Missing an entire subject

In [11]:
missing_ela = df.eperf2.str.isspace() & df.escaleds.str.isspace() & df.ecpi.str.isspace()
missing_ela.sum()

151

In [12]:
missing_math =  df.mperf2.str.isspace() & df.mscaleds.str.isspace() & df.mcpi.str.isspace()
missing_math.sum()

143

In [13]:
missing_science = df.sperf2.str.isspace() & df.sscaleds.str.isspace() & df.scpi.str.isspace()
missing_science.sum()

1521

In [14]:
total_records_with_missing_subject = missing_ela.sum() + missing_math.sum() + missing_science.sum()
total_records_with_missing_subject

1815

## Missing ALL subjects

In [15]:
missing_all_subjects = missing_ela & missing_math & missing_science
missing_all_subjects.sum()

33

In [16]:
df[missing_all_subjects].head()

,sasid,stugrade,escaleds,eperf2,ecpi,mscaleds,mperf2,mcpi,sscaleds,sperf2,scpi
260,1645437,4,,,,,,,,,
459,1400344,5,,,,,,,,,
460,1783711,5,,,,,,,,,
467,1617315,5,,,,,,,,,
514,1374770,5,,,,,,,,,


## Process the data

In [17]:
processor = MCASProcessor()

In [18]:
df_combined = processor.create_combined_subject_dfs(df, subjects=subjects)

In [19]:
# check that each record was copied to each subject df
print(len(df_original) * 3)
print(len(df_combined))

8013
8013


In [20]:
df_no_blanks = processor.remove_rows_with_blankspace_test_scores(df_combined)
len(df_no_blanks)

6198

In [21]:
# check that all blank rows are removed
rows_with_no_scores_removed = len(df_combined) - len(df_no_blanks)
print(rows_with_no_scores_removed)
print(total_records_with_missing_subject)

1815
1815


In [22]:
df_no_blanks.Score1Value.value_counts()

P     2395
NI    1735
A     1192
W      660
       123
F       93
Name: Score1Value, dtype: int64

In [23]:
df_final = processor.correct_performance_level_value(df_no_blanks)

In [24]:
# Make sure only labels changed
df_final.Score1Value.value_counts() == df_no_blanks.Score1Value.value_counts()

4 - P     True
3 - NI    True
5 - A     True
2 - W     True
          True
1 - F     True
Name: Score1Value, dtype: bool

In [25]:
# Re-arrange first two rows for correct format
df_final = processor.reorder_df_columns(df_final, column_list=output_column_format)

In [26]:
# Looks correct
df_final.sample(50)

,NCESID,StudentTestID,StudentLocalID,StudentGradeLevel,TestDate,TestName,TestTypeName,TestSubjectName,TestGradeLevel,Score1Label,Score1Type,Score1Value,Score2Label,Score2Type,Score2Value,Score3Label,Score3Type,Score3Value,Score4Label,Score4Type,Score4Value
404,373737,1897772,,4,4/1/20,MCAS,MCAS ELA,ELA,4,Performance Level,Level,4 - P,Scaled Score,Scale,242,CPI,Scale,100,,,
1764,373737,1565358,,6,5/1/20,MCAS,MCAS Math,Math,6,Performance Level,Level,4 - P,Scaled Score,Scale,256,CPI,Scale,100,,,
750,373737,1664006,,3,4/1/20,MCAS,MCAS ELA,ELA,3,Performance Level,Level,5 - A,Scaled Score,Scale,262,CPI,Scale,100,,,
266,373737,1787615,,4,5/1/20,MCAS,MCAS Math,Math,4,Performance Level,Level,3 - NI,Scaled Score,Scale,220,CPI,Scale,50,,,
487,373737,1588938,,5,6/1/20,MCAS,MCAS Science,Science,5,Performance Level,Level,3 - NI,Scaled Score,Scale,222,CPI,Scale,50,,,
37,373737,1322437,,3,4/1/20,MCAS,MCAS ELA,ELA,3,Performance Level,Level,3 - NI,Scaled Score,Scale,238,CPI,Scale,75,,,
2459,373737,1259913,,10,6/1/20,MCAS,MCAS Science,Science,10,Performance Level,Level,5 - A,Scaled Score,Scale,270,CPI,Scale,100,,,
2427,373737,1657063,,10,5/1/20,MCAS,MCAS Math,Math,10,Performance Level,Level,5 - A,Scaled Score,Scale,264,CPI,Scale,100,,,
2555,373737,1927385,,10,6/1/20,MCAS,MCAS Science,Science,10,Performance Level,Level,5 - A,Scaled Score,Scale,262,CPI,Scale,100,,,
1904,373737,1251434,,7,5/1/20,MCAS,MCAS Math,Math,7,Performance Level,Level,2 - W,Scaled Score,Scale,218,CPI,Scale,25,,,


In [27]:
# Save with different, silly file name
df_final.to_csv(f'{filename.stem}_processorized.csv', index=False)

# !!! Sanity Checks
---

In [28]:
no_first_test_records = df_final.Score1Value.str.isspace()
no_first_test_records.sum()

123

In [29]:
no_second_test_records = df_final.Score2Value.str.isspace()
no_second_test_records.sum()

146

In [30]:
no_third_test_records = df_final.Score3Value.str.isspace()
no_third_test_records.sum()

123

In [31]:
# check that there are no blank rows
(no_first_test_records & no_second_test_records & no_third_test_records).sum()

0

In [32]:
# Some students only have scaled score, data is not randomly missing
no_first_test_records.all() == no_third_test_records.all()

True

# Check original against output
Records chosen by random selection for an example of:
- Completely removing a record with no test scores
- Removing only the subject rows that are missing
- Leaving full records intact

## No missing scores

In [33]:
# no scores missing
df_original.loc[df_original['sasid'] == 1681372]

,sasid,stugrade,escaleds,eperf2,ecpi,mscaleds,mperf2,mcpi,sscaleds,sperf2,scpi
2478,1681372,10,242,P,100,224,NI,50,234,NI,75


In [34]:
# three rows, one per subject
df_combined.loc[df_combined['StudentTestID'] == 1681372]

,StudentTestID,NCESID,StudentLocalID,StudentGradeLevel,TestDate,TestName,TestTypeName,TestSubjectName,TestGradeLevel,Score1Label,Score1Type,Score1Value,Score2Label,Score2Type,Score2Value,Score3Label,Score3Type,Score3Value,Score4Label,Score4Type,Score4Value
2478,1681372,373737,,10,4/1/20,MCAS,MCAS ELA,ELA,10,Performance Level,Level,P,Scaled Score,Scale,242,CPI,Scale,100,,,
2478,1681372,373737,,10,5/1/20,MCAS,MCAS Math,Math,10,Performance Level,Level,NI,Scaled Score,Scale,224,CPI,Scale,50,,,
2478,1681372,373737,,10,6/1/20,MCAS,MCAS Science,Science,10,Performance Level,Level,NI,Scaled Score,Scale,234,CPI,Scale,75,,,


In [35]:
# no rows removed
df_final.loc[df_final['StudentTestID'] == 1681372]

,NCESID,StudentTestID,StudentLocalID,StudentGradeLevel,TestDate,TestName,TestTypeName,TestSubjectName,TestGradeLevel,Score1Label,Score1Type,Score1Value,Score2Label,Score2Type,Score2Value,Score3Label,Score3Type,Score3Value,Score4Label,Score4Type,Score4Value
2478,373737,1681372,,10,4/1/20,MCAS,MCAS ELA,ELA,10,Performance Level,Level,4 - P,Scaled Score,Scale,242,CPI,Scale,100,,,
2478,373737,1681372,,10,5/1/20,MCAS,MCAS Math,Math,10,Performance Level,Level,3 - NI,Scaled Score,Scale,224,CPI,Scale,50,,,
2478,373737,1681372,,10,6/1/20,MCAS,MCAS Science,Science,10,Performance Level,Level,3 - NI,Scaled Score,Scale,234,CPI,Scale,75,,,


## One score missing

In [36]:
# original has two test scores
df_original.loc[df_original['sasid'] == 1760810]

,sasid,stugrade,escaleds,eperf2,ecpi,mscaleds,mperf2,mcpi,sscaleds,sperf2,scpi
2669,1760810,7,,W,100,,W,100,,,


In [37]:
# three rows, one per subject
df_combined.loc[df_combined['StudentTestID'] == 1760810]

,StudentTestID,NCESID,StudentLocalID,StudentGradeLevel,TestDate,TestName,TestTypeName,TestSubjectName,TestGradeLevel,Score1Label,Score1Type,Score1Value,Score2Label,Score2Type,Score2Value,Score3Label,Score3Type,Score3Value,Score4Label,Score4Type,Score4Value
2669,1760810,373737,,7,4/1/20,MCAS,MCAS ELA,ELA,7,Performance Level,Level,W,Scaled Score,Scale,,CPI,Scale,100,,,
2669,1760810,373737,,7,5/1/20,MCAS,MCAS Math,Math,7,Performance Level,Level,W,Scaled Score,Scale,,CPI,Scale,100,,,
2669,1760810,373737,,7,6/1/20,MCAS,MCAS Science,Science,7,Performance Level,Level,,Scaled Score,Scale,,CPI,Scale,,,,


In [38]:
# blank row removed
df_final.loc[df_final['StudentTestID'] == 1760810]

,NCESID,StudentTestID,StudentLocalID,StudentGradeLevel,TestDate,TestName,TestTypeName,TestSubjectName,TestGradeLevel,Score1Label,Score1Type,Score1Value,Score2Label,Score2Type,Score2Value,Score3Label,Score3Type,Score3Value,Score4Label,Score4Type,Score4Value
2669,373737,1760810,,7,4/1/20,MCAS,MCAS ELA,ELA,7,Performance Level,Level,2 - W,Scaled Score,Scale,,CPI,Scale,100,,,
2669,373737,1760810,,7,5/1/20,MCAS,MCAS Math,Math,7,Performance Level,Level,2 - W,Scaled Score,Scale,,CPI,Scale,100,,,


## All scores missing

In [39]:
# no test scores
df_original.loc[df_original['sasid'] == 1465689]

,sasid,stugrade,escaleds,eperf2,ecpi,mscaleds,mperf2,mcpi,sscaleds,sperf2,scpi
1569,1465689,8,,,,,,,,,


In [40]:
# three rows, one per subject
df_combined.loc[df_combined['StudentTestID'] == 1465689]

,StudentTestID,NCESID,StudentLocalID,StudentGradeLevel,TestDate,TestName,TestTypeName,TestSubjectName,TestGradeLevel,Score1Label,Score1Type,Score1Value,Score2Label,Score2Type,Score2Value,Score3Label,Score3Type,Score3Value,Score4Label,Score4Type,Score4Value
1569,1465689,373737,,8,4/1/20,MCAS,MCAS ELA,ELA,8,Performance Level,Level,,Scaled Score,Scale,,CPI,Scale,,,,
1569,1465689,373737,,8,5/1/20,MCAS,MCAS Math,Math,8,Performance Level,Level,,Scaled Score,Scale,,CPI,Scale,,,,
1569,1465689,373737,,8,6/1/20,MCAS,MCAS Science,Science,8,Performance Level,Level,,Scaled Score,Scale,,CPI,Scale,,,,


In [41]:
# all rows removed
df_final.loc[df_final['StudentTestID'] == 1465689]

,NCESID,StudentTestID,StudentLocalID,StudentGradeLevel,TestDate,TestName,TestTypeName,TestSubjectName,TestGradeLevel,Score1Label,Score1Type,Score1Value,Score2Label,Score2Type,Score2Value,Score3Label,Score3Type,Score3Value,Score4Label,Score4Type,Score4Value


## What about the duplicate records?

In [42]:
# five test scores between two students
df_original.loc[df_original['sasid'] == 1840155]

,sasid,stugrade,escaleds,eperf2,ecpi,mscaleds,mperf2,mcpi,sscaleds,sperf2,scpi
1363,1840155,7,254,P,100,258,P,100,,,
2450,1840155,10,268,A,100,264,A,100,262,A,100


In [43]:
# six rows
df_combined.loc[df_combined['StudentTestID'] == 1840155]

,StudentTestID,NCESID,StudentLocalID,StudentGradeLevel,TestDate,TestName,TestTypeName,TestSubjectName,TestGradeLevel,Score1Label,Score1Type,Score1Value,Score2Label,Score2Type,Score2Value,Score3Label,Score3Type,Score3Value,Score4Label,Score4Type,Score4Value
1363,1840155,373737,,7,4/1/20,MCAS,MCAS ELA,ELA,7,Performance Level,Level,P,Scaled Score,Scale,254,CPI,Scale,100,,,
2450,1840155,373737,,10,4/1/20,MCAS,MCAS ELA,ELA,10,Performance Level,Level,A,Scaled Score,Scale,268,CPI,Scale,100,,,
1363,1840155,373737,,7,5/1/20,MCAS,MCAS Math,Math,7,Performance Level,Level,P,Scaled Score,Scale,258,CPI,Scale,100,,,
2450,1840155,373737,,10,5/1/20,MCAS,MCAS Math,Math,10,Performance Level,Level,A,Scaled Score,Scale,264,CPI,Scale,100,,,
1363,1840155,373737,,7,6/1/20,MCAS,MCAS Science,Science,7,Performance Level,Level,,Scaled Score,Scale,,CPI,Scale,,,,
2450,1840155,373737,,10,6/1/20,MCAS,MCAS Science,Science,10,Performance Level,Level,A,Scaled Score,Scale,262,CPI,Scale,100,,,


In [44]:
# one row removed == five test scores
df_final.loc[df_final['StudentTestID'] == 1840155]

,NCESID,StudentTestID,StudentLocalID,StudentGradeLevel,TestDate,TestName,TestTypeName,TestSubjectName,TestGradeLevel,Score1Label,Score1Type,Score1Value,Score2Label,Score2Type,Score2Value,Score3Label,Score3Type,Score3Value,Score4Label,Score4Type,Score4Value
1363,373737,1840155,,7,4/1/20,MCAS,MCAS ELA,ELA,7,Performance Level,Level,4 - P,Scaled Score,Scale,254,CPI,Scale,100,,,
2450,373737,1840155,,10,4/1/20,MCAS,MCAS ELA,ELA,10,Performance Level,Level,5 - A,Scaled Score,Scale,268,CPI,Scale,100,,,
1363,373737,1840155,,7,5/1/20,MCAS,MCAS Math,Math,7,Performance Level,Level,4 - P,Scaled Score,Scale,258,CPI,Scale,100,,,
2450,373737,1840155,,10,5/1/20,MCAS,MCAS Math,Math,10,Performance Level,Level,5 - A,Scaled Score,Scale,264,CPI,Scale,100,,,
2450,373737,1840155,,10,6/1/20,MCAS,MCAS Science,Science,10,Performance Level,Level,5 - A,Scaled Score,Scale,262,CPI,Scale,100,,,
